In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator #augment  images
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint # to get a view on internal states and statistics of the model during training
#define where to checkpoint the model weights, how to name the file
from keras.applications.vgg16 import VGG16 #Very Deep Convolutional Networks
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

import warnings
warnings.filterwarnings('ignore')

In [2]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

In [3]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False

In [4]:
img_width, img_height = 150, 150
train_data_dir = r"data\train"
val_data_dir = r"C:data\validation"
model_weights_file = 'vgg16-xfer-weights.h5'
nb_train_samples = 460
nb_val_samples = 55
nb_epochs = 5

In [5]:
input = Input(shape=(img_width, img_height, 3))
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(input, x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [21]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
#augment  images
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height),
                                                    batch_size=32, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height),
                                                        batch_size=32,class_mode='categorical')

Found 460 images belonging to 2 classes.
Found 55 images belonging to 2 classes.


In [7]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, epochs=5, callbacks = callbacks, validation_data=validation_generator)
print('Training Completed!')

Epoch 1/5
15/15 [==============================] - 37s 2s/step - loss: 1.2625 - accuracy: 0.6065 - val_loss: 0.6660 - val_accuracy: 0.6182
Epoch 2/5
15/15 [==============================] - 38s 3s/step - loss: 0.4900 - accuracy: 0.7652 - val_loss: 0.4954 - val_accuracy: 0.6909
Epoch 3/5
15/15 [==============================] - 46s 3s/step - loss: 0.4813 - accuracy: 0.7413 - val_loss: 0.6074 - val_accuracy: 0.6909
Epoch 4/5
15/15 [==============================] - 46s 3s/step - loss: 0.4119 - accuracy: 0.8217 - val_loss: 0.5657 - val_accuracy: 0.6727
Epoch 5/5
15/15 [==============================] - 47s 3s/step - loss: 0.3718 - accuracy: 0.8391 - val_loss: 0.5231 - val_accuracy: 0.7818
Training Completed!


In [22]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

#img_path = r"data\test\adidasShoe.jpg"

img_path=r"data\test\nikeShoe.jpg"

label = ['Nike', 'Adidas']

img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print('Input Shape:', x.shape)

features = model.predict(x)

print('Predicted Array:', features)

if features.size > 0:
    ind = np.argmax(features)  # Using np.argmax instead of np.where
    print('Predicted Label:', label[ind])
else:
    print('No predictions were made.')

Input Shape: (1, 150, 150, 3)
1/1 [==============================] - 0s 205ms/step
Predicted Array: [[1.00000e+00 3.67867e-15]]
Predicted Label: Nike
